In [ ]:
import sys, os, math
from scipy import stats
import numpy as np
import pandas as pd

from statistics import mode

from scipy import stats

# sys.path.insert(1, '../libs/')
# from stat_lib import *

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

### Comparando-se o mesmo grupo duas vezes (pareado)

Vamos supor que temos um grupo de pacientes que vão ser medicados. Medimos um determinado marcador (p.ex. glicemia) antes da medicação e depois da medicação. ** Desenho experimental e cálculo de número amostral (N).

A principal pergunta: - O medicmaento fez efeito

Caso a distribuição do biomarcador seja normal, podemos fazer o teste t-student para comparar os dois grupos.

Glicose no sangue (mg/dl)
  - normal: < 99 mg/dl
  - pré-diabético: [100, 125] mg/dl
  - diabético: > 125 mg/dl

In [ ]:
def criar_dados_normais(MU, SSD, N):
    samp = np.random.normal(loc=MU, scale=SSD, size=N)
    return samp

# metodo estatístico
def calc_estatistica_descritiva(lista: list, verbose:bool=False):

    N = len(lista)
	
    mini = np.min(lista)
    maxi = np.max(lista)

    mu = np.mean(lista)
    med = np.median(lista)
    mod = mode(lista)

    ssd = np.std(lista)

    n = len(lista)
    n4 = int(n/4)

    lista = list(lista)
    lista.sort()

    mini = int( lista[0] )
    maxi = int( lista[-1] )

    q1, q2, q3 = np.quantile(lista, [0.25, 0.5, 0.75])

    s_quantile = f"mínimo {mini}, Q1 {q1}, mediana {q2}, Q3 {q3}, máximo {maxi}"

    stri = f"N={N}, média={mu:.2f} mediana={q2:.2f} moda={mod:.2f} e ssd={ssd:.2f}"
    if verbose:
      print(stri)
      print(s_quantile)

    return n, mu, q1, q2, q3, mod, ssd, mini, maxi, stri, s_quantile

def calc_tamanho_efeito(n1:int, mu1:float, ssd1:float, 
                        n2:int, mu2:float, ssd2:float, verbose:bool=False):

    ssd_pool = math.sqrt( ((n1-1)*ssd1**2 + (n2-1)*ssd2**2) / (n1+n2-2) )
    diff = mu2-mu1
    
    EffSize = diff / ssd_pool
    stri = f"O tamanhao de efeito é de {EffSize:.2f} para uma diferença de {diff:.2f} e o ssd conjunto de {ssd_pool:.2f}"
    if verbose:
      print(stri)

    return EffSize, diff, ssd_pool, stri

def stat_asteristics(pval, NS='NS'):
    if pval >= 0.05:   return NS
    if pval > 0.01:    return '*'
    if pval > 0.001:   return '**'
    if pval > 0.0001:  return '***'
    return '****'

def calc_ttest_independente(samp1:list, samp2:list, equal_var:bool=True, alpha:float=0.05):

    # t-test independente
    t, pval = stats.ttest_ind(samp2, samp1, equal_var=equal_var)

    msg = f"Teste-t independente = estatística {t:.3f} e p-valor {pval:.2e}"

    if pval >= alpha:
        msg += '\n' + "Aceitamos a Hipótese nula, não houve efeicalc_normalidade_SWTto significativo."
    else:
        msg += '\n' + "Rejeitamos a Hipótese nula, houve efeito significativo."

    return t, pval, msg   
    
def calc_normalidade_SWT(sample, alpha = 0.05, NS='NS'):
    # teste de normalidade de Shapiro-Wilkis
    stat, pvalue = stats.shapiro(sample)

    if pvalue > alpha:
        text = 'Segundo o teste de Shapiro-Wilk a distribuição se assemelha a uma distribuição normal (aceita-se H0)'
        ret = True
    else:
        text = 'Segundo o teste de Shapiro-Wilk a distribuição não se assemelha a uma distribuição normal (rejeita-se H0)'
        ret = False

    s_ater = stat_asteristics(pvalue)
    text_stat = f'p-value {pvalue:.2e} ({s_ater})'

    return ret, text, text_stat, stat, pvalue
    
def plot_2_distribuições_normais(MUs, SSDs, N1, N2, colors = ['blue', 'red'], figsize=(9, 6)):

    assert len(MUs)==2, 'Enviar 2 médias'
    assert isinstance(MUs, list), 'Enviar 2 médias na forma lista'

    assert len(SSDs)==2, 'Enviar 2 desvios padrões amostrais'
    assert isinstance(SSDs, list), 'Enviar 2 desvios padrões amostrais na forma lista'

    MU1, SSD1 = MUs[0], SSDs[0]
    MU2, SSD2 = MUs[1], SSDs[1]
    
    samp1 = criar_dados_normais(MU1, SSD1, N1)
    samp2 = criar_dados_normais(MU2, SSD2, N2)
    
    n1, mu1, q11, med1, q21, mod1, ssd1, mini1, maxi1, stri1, s_quantile1, = \
    calc_estatistica_descritiva(samp1, verbose=False)
    
    n2, mu2, q12, med2, q22, mod2, ssd2, mini2, maxi2, stri2, s_quantile2, = \
    calc_estatistica_descritiva(samp2, verbose=False)

    ES, diff_ES, ssd_pool, stri_ES = calc_tamanho_efeito(n1, mu1, ssd1, n2, mu2, ssd2)

    fig, ax = plt.subplots(figsize=figsize)

    title = ''

    for i in range(2):
        color = colors[i]

        label = 'dist%d'%(i+1)

        if i == 0:
            mu, ssd, n = mu1, ssd1, n1
            samples = samp1
        else:
            mu, ssd, n = mu2, ssd2, n2
            samples = samp2

        
        ax = sns.histplot(samples, stat='density', color=color, alpha=0.3, label=label, ax=ax)
        sns.rugplot(samples, color=color, alpha=0.1, ax=ax)

        max_y = max(p.get_height() for p in ax.patches)
    
        # Criando o eixo x        
        seqx = np.linspace(stats.norm.ppf(0.001, MUs[i], SSDs[i]), stats.norm.ppf(0.999, MUs[i], SSDs[i]), 100)
    
        # fiting da curva teorica
        normal_pdf = stats.norm.pdf(seqx, MUs[i], SSDs[i])
        sns.lineplot(x=seqx, y=normal_pdf, color=colors[i])

        
        ax.axvline(x=mu, ymin=0, ymax=max_y, color=color)
        ax.axvline(x=mu+ssd, ymin=0, ymax=max_y/2, color=color, linestyle='--')
        ax.axvline(x=mu-ssd, ymin=0, ymax=max_y/2, color=color, linestyle='--')
    
        ret, text, text_stat, stat, pvalue = calc_normalidade_SWT(samples, alpha = 0.05, NS='NS')

        if title != '':
            title += '\n'

        title += f"A distribuição {i+1} ({color}) tem media(SSD) = {mu:.2f} ({ssd:.2f}) e N={n}"
        title += '\n' + text + ' -> ' + text_stat
    
    title += f'\nTamanho de efeito: {ES:.2f}, diferença={diff_ES:.2f}, ssd conjunto={ssd_pool:.2f}'

    diff_ssd = np.abs(ssd1 - ssd2)
    equal_var = diff_ssd <= ssd1*.50
    t, pval, ttest_msg  = calc_ttest_independente(samp1, samp2, equal_var=equal_var, alpha=0.05)

    title += f'\nt-test independente: {ttest_msg}'

    ax.set_xlabel("values")
    ax.set_ylabel("percentage (%)")
    ax.set_title(title)
    plt.grid()
    plt.legend();


In [ ]:
MU1 = 140; SSD1 = 15
MU2 = 110; SSD2 = 25

MUs = [MU1, MU2]
SSDs = [SSD1, SSD2]
N = 30
N1=N
N2=N

samp1 = criar_dados_normais(MU1, SSD1, N1)
samp2 = criar_dados_normais(MU2, SSD2, N2)

n1, mu1, q11, med1, q21, mod1, ssd1, mini1, maxi1, stri1, s_quantile1, = \
calc_estatistica_descritiva(samp1, verbose=False)

n2, mu2, q12, med2, q22, mod2, ssd2, mini2, maxi2, stri2, s_quantile2, = \
calc_estatistica_descritiva(samp2, verbose=False)

print("amostra 1", stri1)
print("amostra 2", stri2)

In [ ]:
mu1, mu2

### As distribuições são nomais? teste de Shapiro-Wilkis

In [ ]:
ret, text, text_stat, stat, pvalue = calc_normalidade_SWT(samp1, alpha = 0.05, NS='NS')
print("Amostra 1", text)
print(text_stat)

In [ ]:
ret, text, text_stat, stat, pvalue = calc_normalidade_SWT(samp2, alpha = 0.05, NS='NS')
print("Amostra 2", text)
print(text_stat)

### Teste de Shapiro-Wilk

In [ ]:
ret1, text1, text_stat1, stat1, pvalue1 = calc_normalidade_SWT(samp1, 0.05, NS='---')
text1, text_stat1

In [ ]:
ret2, text2, text_stat2, stat2, pvalue2 = calc_normalidade_SWT(samp2, 0.05, NS='---')
text2, text_stat2

### As variâncias iguais?
  - dizemos que sim se a diferença entre elas (valor absoluto) for menor que 50% de ssd1

In [ ]:
plot_2_distribuições_normais(MUs, SSDs, N1, N2, colors = ['blue', 'red'], figsize=(12, 6))

### Como entendemos este resultado?

  - Controle: pacientes não medicados
  - Case:     pacientes medicados
  
  
  - Se distriuições similares (t-test, diferença próxima a zero - e o intervalo de confiança passa pelo zero) significa que a droga não fez efeito
  - Se distriuições distantes (t-test, intervalo de confiança (IC) longe do zero) significa que a droga fez efeito
    - ter um efeito DESASTROSO, neste caso, se a diferença for positiva
    - ter um efeito BENÉFICO  , neste caso, se a diferença for negativa
  

### O quão distante estão as distribuições???

  - Vamos calcular:
    - Intervalo de confiança
    - Tamanho do efeito (effect size)
    - A diferença entre as médias
    - Mostrar como se apresenta estes resultados

### Relembrando conceitos

#### Teste t

statistician William Sealy Gosset 

<font size="4">t-stat = $\frac{<X>-\mu}{\frac{SSD}{\sqrt(n)}}$</font>

de forma mais generalizada dadas duas distribuições X1 e X2

<font size="4">t-stat = $\frac{<X1>-<X2>}{\sqrt( \frac{var1}{n1} + \frac{var2}{n2}) }$</font>


#### Tamanho do efeito (effect size)

<font size="4">$ES = \frac{media_2 - media_1}{SSD_{pool}}$</font>

https://en.wikipedia.org/wiki/Effect_size


#### Distância entre médias (pode ser negativo)

<font size="4">$diff = media_2 - media_1$</font>


#### SEM - Standard Error of the mean

<font size="4">$SEM = \frac{SSD}{\sqrt{n}}$</font>

